In [22]:
from forex_python.converter import CurrencyRates
from smart_open import open
from datetime import datetime, timedelta
import requests
import io
from io import StringIO
import os

In [ ]:
"""
b = 'gs://pd-fxrates'
f = '/to/data.csv'
url_endpoint = 'http://www.bankofengland.co.uk/boeapps/iadb/fromshowcolumns.asp?csv.x=yes'
trg_f = 'f.csv'
day_offset = 1
"""

In [26]:
b = '.'
f = '/data.csv'
url_endpoint = 'http://www.bankofengland.co.uk/boeapps/iadb/fromshowcolumns.asp?csv.x=yes'
trg_f = 'f.csv'
day_offset = 1

In [67]:
#def fx_get(event, context):
if 1==1:
  #print(event['name'])
  #if event['name'] != trg_f:
  #  return

  yest = datetime.today().date() - timedelta(days = day_offset)
  s_yest = yest.strftime("%d/%m/%Y")
  s_dtfrom = yest.strftime("%d/%b/%Y")
  s_dtto = s_dtfrom
  """
  c = CurrencyRates()
  xe = c.get_rate('EUR', 'USD', yest)
  """
  xe = 1.0588
  #selects the series to be loaded
  payload = {
    'Datefrom'   : s_dtfrom,
    'Dateto'     : s_dtto,
    'SeriesCodes': 'XUDLERD',
    'CSVF'       : 'TN',
    'UsingCodes' : 'Y',
    'VPD'        : 'Y',
    'VFD'        : 'N'
  }
  #loads the time series
  #response = requests.get(url_endpoint, params=payload)
  #s_boe = io.BytesIO(response.content).read().decode('UTF-8')
  s_boe = """DATE,XUDLERD\n26 Mar 2020,0.9072"""
  #s_boe = 'DATE,XUDLERD'

  s_boe_xe = s_boe.split(',')[-1]

  # BOE does not publish for non trading days, get ready to fill with the previous value if needed
  # csv read is necessary later anyway for appending the new line because GCS files immutable
  with open(b + f) as fin:
    csv = fin.read()
  
  # alt to below check is isinstance(yourNumber, numbers.Real)
  if s_boe_xe.replace('.','') .isnumeric():
    xe_boe = str(round(1 / float(s_boe_xe), 4))
  else:
    lines = csv.split('\n')
    xe_boe = lines[-2].split(',')[2]
  
  
  #Date,FX,BOE_FX
  l = ','.join([s_yest, str(xe), xe_boe + '\n'])

  with open(b + f, 'w') as fout:
    fout.write(csv + l)

In [79]:
csv = """06/01/2017,1.0586,1.0576
07/01/2017,1.0587,1.0577
"""
lines = csv.split('\n')
#len(lines)
lines[-2].split(',')[2]

'1.0577'

In [68]:
with open(b + f) as fin:
    print(fin.read())

06/01/2017,1.0586,1.0576
07/01/2017,1.0587,1.0577
08/01/2017,1.0588,1.0578
26/03/2020,1.0588,1.0576
27/03/2020,1.0588,1.0576
27/03/2020,1.0588,1.1023

